# 3. A primer on adaptive speciation

*AdaScape* is a python package with routines to predict the ecological and evolutionary dynamics of individuals as their environment changes at larger (hundreds to thousands of kilometres) and longer (thousand to millions of years) scales. The adaptive speciation model presented here build on the *adaptive dynamics* theoretical framework ([Dieckmann et al. 2007](https://link.springer.com/chapter/10.1007/978-3-540-34428-5_8); [Brännström et al. 2012](https://www.evolutionary-ecology.com/abstracts/v14/2747.html); [McGill & Brown 2012](https://www.annualreviews.org/doi/10.1146/annurev.ecolsys.36.091704.175517)), which focus on the evolution of a trait (or various traits) and is affected by processes of **environmental selection, mutation, dispersal, and ecological interactions**. In particular, we have modified the simple speciation model proposed by [Irwin 2012](https://www.journals.uchicago.edu/doi/10.1086/666002) to consider a) more than trait, b)competion for a limiting resrouce, and c) a trait definition that allow us to reconstruct phylogenetic trees. 

The first of these evolutionary processes is **environmental selection**, which describes the environmental fitness $f_i$ of an individual $i$ with a vector of trait values $\mathbf{u}_{i} = (u_{i, 1}, u_{i, 2}, \ldots, u_{i, k})^{\intercal}$. The environmental fitness is given by a multivariate Gausian function as:

$f_i (\mathbf{u}_{i}) = \exp{\left(-\dfrac{1} {2}  \left(\mathbf{u}_{i} - \mathbf{u}_{0}(z_i)\right)^\intercal \cdot \mathbf{\Sigma}^{-1} \cdot \left(\mathbf{u}_{i}- \mathbf{u}_{0}(z_i)\right) \right)}$,

<img src="./schematic_fitness_multinorm.png" alt="Enviromental_fitness" width="300"/>

where $u_{0}(z_i)$ is the optimal trait value vector for a given local environmental condition $z_i$ and $\mathbf{\Sigma}$ is the $k \times k$ matrix of parameters driving the fitness changes. The diagonal of $\mathbf{\Sigma}$ is given by the parameter **$\mathbf{\sigma_f}$ is the environmental fitness variability** that determines the strength of the selection of an individual's trait value against its optimum. We define the optimal trait value as a linear relationship between the optimal trait and an environmental gradient as:

$u_{0, q}(z_i) = \alpha_{z} \cdot \left(Z_i(z_i) - \dfrac{1}{2}\right) + \dfrac{1}{2}, \, \forall\, q = 1, \ldots, k$,

where $\alpha_z$ is the slope of the linear regression and $Z_i$ is the normalized environmental conditions experience by individual $i$. Here we normalized the environmental field from the maximum $z_{max}$ and minimum $z_{min}$ ranges that this field can reach during a simulation and is expressed as: 

$Z_i(z_i)  = \frac{z_i - z_{min}}{z_{max} - z_{min}}$.

<img src="./schematic_env_filter.png" alt="Enviromental_filter" width="300"/>

The second evolutionary process is **mutation**, which is described for each individual as a random process. For this we take a random number drawn from a uniform distribution $\mathcal{U}(0,1)$ and compare it with a mutation probability $p_m$, if the drawn number is less than $p_m$ then the descendent of the individual $i$ can mutate to a trait value taken from a gaussian distribution $\mathcal{N}(u_{i, q}, \sigma_m)$ centred at the ancestor trait value $u_{i,q}$ and with a **mutation variability $\mathbf{\sigma_m}$**.

<img src="./schematic_mutation.png" alt="Mutation" width="400"/>

The third evolutionary process we consider is **dispersal**, where the new location of an individual $i$ is randomly sampled around the position of each individual $l_{i, x}$ and $l_{i, y}$, along the X and Y axes using separated Gaussian distribution $\mathcal{N}(l_{i,\bullet}, \sigma_d)$, where $\bullet$ is a place holder for x or y. In the next time step the individual will then move to a new location, where the amount of spatial displacement is determined by the parameter **dispersal variability** $\mathbf{\sigma_d}$ 

<img src="./schematic_dispersal.png" alt="Dispersal" width="800"/>

The ecological interaction we consider in *AdaScape* is via the trait-mediated competition for a single a resource in the local neighbourhood $D_{i, r}$ centred at individual $i$ and with radius $r$:

$n_{eff, i} = \sum_{j \in D_{i,r} }{\exp{\left( -\frac{\Delta_{u}^2(i,j)} {2\sigma_{u}^2} \right)} }$, 

\noindent where $\Delta_{u}(i,j)$ is the trait distance between individual $i$ and its neighbour $j$, and $\sigma_u$ is **trait-distance variability**. The local neighbourhood is the area around each individual centred at its location $l_{i, \bullet}$ and with an extend determined by $r$, and can be thought of as the area where local competition for resources take place. The parameter, $\sigma_u$, dictates the strength of the competition among individuals with similar trait values. Hence if $\sigma_u \geq 1$, all individuals in the local neighbourhood regardless of their trait values are assumed to compete for the same resource. However, if $\sigma_u <1$ only those individuals with similar trait values to the focus individual $i$ are counted and thus assumed to compete for the same resource.

<img src="./schematic_comp.png" alt="Competition" width="300"/>

The temporal dynamics for the speciation model without competition ($\sigma_u =2$)  and with competition ($\sigma_u =0.2$) are shown over a schematic depicting the generalities of the adaptive speciation models along an elevation and a rainfall gradient and where the grey arrows illustrate dispersal and the red arrows competition strength.

<img src="./schematic_top.png" alt="Enviromental_top" width="1100"/>

The model is implemented as an individual-based, spatially-explicit model. A simulation is initialised with a given number of individuals allocated randomly or at a particular range in a continuous 2D space. The traits for each individual are drawn from a uniform distribution, where the minimum and maximum range is between 0 and 1. After initialisation, the fitness for each individual is evaluated following equation above. Then we compute the number of offspring $n_{off, i}$ for each individual $i$ following [Irwin (2012)](https://www.journals.uchicago.edu/doi/10.1086/666002):

$n_{off, i} = \frac{K}{n_{eff,i}}  f_i,$

where $K$ is the carrying capacity and thus $\frac{K}{n_{eff,i}}$ represents the density dependent reproductive factor. After the number of offspring has been determined, the new individuals are generated, mutated, and dispersed as explained above. 

## Taxon definition

We define a taxon as a group of individuals sharing similar trait values and common ancestry (sensu [Pontarp & Wiens](https://onlinelibrary.wiley.com/doi/abs/10.1111/jbi.12896)). We implement this by using a spectral clustering algorithm [Luxburg2007](), which examines individuals per ancestor group and assigns them to new taxon groups based on the clustering of their trait values at each time step. To each of these clusters we assigned a new taxon-id $\in \mathbb{N}= \{0,1,2,3, \ldots, \infty\}$, this taxon-id at time $t$ will become the ancestor-id at time $t+1$:

<img src="./taxon_def.png" alt="Taxon_definition" width="700"/>

More information and examples of AdaScape please refer to our manuscript [GMD](https://gmd.copernicus.org/preprints/gmd-2023-72/) or in our repository in [GitHub](https://github.com/fastscape-lem/adascape).